In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 25823586
paper_name = 'zhu_li_2015' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Zhu el al G3_qCTF_HI Summary.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5263 x 4


In [7]:
original_data.head()

,Plate,Position,ORF,CIN Rate
0,1,A01,Control681,0.000074
1,1,A02,YAL068C,0.000070
2,1,A03,YAL067C,0.000065
3,1,A04,YAL066W,0.000111
4,1,A05,YAL065C,0.000095


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
original_data.loc[original_data['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [14]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [15]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

             Plate Position                     ORF  CIN Rate  \
index_input                                                     
0                1      A01              Control681  0.000074   
96               2      A02              Control681  0.000053   
192              3      A03              Control681  0.000053   
288              4      A04              Control681  0.000055   
384              5      A05              Control681  0.000061   
...            ...      ...                     ...       ...   
5256            75       H6                     NaN       NaN   
5257            75       H7                     NaN       NaN   
5258            75       H8                     NaN  0.000000   
5259            75       H9  parental-strain-BY4730       NaN   
5260            75      H10  parental-strain-BY4741  0.000000   

                                orf  
index_input                          
0                        CONTROL681  
96                       CONTROL681  
19

In [17]:
ctrl_avg = original_data.loc[original_data['orf'].str.contains('CONTROL'),'CIN Rate'].mean()

In [19]:
original_data = original_data.loc[t,:]

In [20]:
original_data['data'] = original_data['CIN Rate'] / ctrl_avg

In [21]:
original_data.set_index('orf', inplace=True)

In [22]:
original_data = original_data[['data']].copy()

In [23]:
original_data = original_data.groupby(original_data.index).mean()

In [24]:
original_data.shape

(4944, 1)

# Prepare the final dataset

In [25]:
data = original_data.copy()

In [26]:
dataset_ids = [16006]
datasets = datasets.reindex(index=dataset_ids)

In [27]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [28]:
data.head()

dataset_id,16006
data_type,value
orf,
YAL002W,0.968204
YAL004W,0.958738
YAL005C,1.602038
YAL007C,1.267836
YAL008W,1.282876


## Subset to the genes currently in SGD

In [29]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [30]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16006
,data_type,value
gene_id,orf,
2,YAL002W,0.968204
1863,YAL004W,0.958738
4,YAL005C,1.602038
5,YAL007C,1.267836
6,YAL008W,1.282876


# Normalize

In [31]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [32]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [33]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          16006          
data_type           value    valuez
gene_id orf                        
2       YAL002W  0.968204  0.047459
1863    YAL004W  0.958738  0.036442
4       YAL005C  1.602038  0.785108
5       YAL007C  1.267836  0.396167
6       YAL008W  1.282876  0.413671

# Print out

In [34]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [35]:
from IO.save_data_to_db3 import *

In [36]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 25823586...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.23s/it]

Updating the data_modified_on field...
